
# 1. Préparation du corpus


Notre "corpus" est une petite phrase.

Dans un vrai modèle de langage, on aurait des milliers / millions de phrases.

In [2]:

text = "les modèles de langage les modèles statistiques les modèles neuronaux"

# On découpe la phrase en mots (tokens) en séparant sur les espaces.
tokens = text.split()

print("Tokens :", tokens)
print("Nombre de tokens :", len(tokens))


Tokens : ['les', 'modèles', 'de', 'langage', 'les', 'modèles', 'statistiques', 'les', 'modèles', 'neuronaux']
Nombre de tokens : 10



# 2. Construction du modèle bigramme

- Ce qu’est un bigramme : p(w2∣W1)

- Que bigrams 'les' contient tous les mots qui ont suivi “les” dans le corpus → approximation empirique de la distribution.

- On ne calcule pas les probabilités, mais on les approxime en tirant au hasard parmi ces listes


In [3]:

from collections import defaultdict

# 'bigrams' va associer à chaque mot la liste des mots qui peuvent le suivre
# Exemple attendu :
# bigrams["les"] = ["modèles", "modèles", "modèles"]
bigrams = defaultdict(list)

# zip(tokens[:-1], tokens[1:]) permet de créer des paires (w1, w2)
# (les, modèles), (modèles, de), (de, langage), ...
for w1, w2 in zip(tokens[:-1], tokens[1:]):
    bigrams[w1].append(w2)

# Visualisation du modèle appris
print("Modèle bigramme (mot -> mots suivants possibles) :\n")
for w, next_words in bigrams.items():
    print(f"{w:10s} -> {next_words}")


Modèle bigramme (mot -> mots suivants possibles) :

les        -> ['modèles', 'modèles', 'modèles']
modèles    -> ['de', 'statistiques', 'neuronaux']
de         -> ['langage']
langage    -> ['les']
statistiques -> ['les']


# 3 Fonction de génération

- On ne prédit pas des lettres, mais des mots entiers (modèle au niveau des mots).

- Modèle très limité : il ne regarde que le mot précédent → mémoire très courte.

- On ne calcule pas explicitement les probabilités, mais l’échantillonnage aléatoire parmi les successeurs est cohérent avec la fréquence.

In [5]:


import random

def generate(start="les", length=8):
    """
    Génère une séquence de mots en utilisant le modèle bigramme.

    Paramètres
    ----------
    start : str
        Mot de départ de la séquence.
    length : int
        Longueur totale souhaitée (nombre de mots).

    Retour
    ------
    str
        Texte généré.
    """
    seq = [start]  # on commence par le mot de départ

    for _ in range(length - 1):
        current = seq[-1]  # dernier mot généré

        # Liste des mots possibles qui peuvent suivre 'current'
        candidats = bigrams.get(current)

        if not candidats:
            # Si on n'a jamais vu ce mot comme "w1" dans le corpus,
            # on casse la génération (ou on pourrait relancer depuis "les").
            # Ici on choisit de s'arrêter.
            break

        # Choix aléatoire parmi les mots suivants observés dans le corpus
        next_word = random.choice(candidats)
        seq.append(next_word)

    return " ".join(seq)

# Test simple
print(generate())


les modèles de langage les modèles de langage



# 4. Expérimenter le modèle

In [6]:


for i in range(5):
    print(f"Phrase générée {i+1} : {generate(start='les', length=8)}")


Phrase générée 1 : les modèles neuronaux
Phrase générée 2 : les modèles de langage les modèles neuronaux
Phrase générée 3 : les modèles neuronaux
Phrase générée 4 : les modèles de langage les modèles de langage
Phrase générée 5 : les modèles de langage les modèles de langage


 # Remarques
    
- Les phrases générées sont grammaticalement simples, souvent répétitives.

- Le modèle ne sait pas “terminer” une phrase, ni gérer la syntaxe globale.

- Il ne capture aucune structure longue distance (sujet / verbe / complément, accord, etc.).

- C’est une première étape pour comprendre ce que les LLMs font… mais à une échelle gigantesque et avec des architectures complexes (Transformers, embeddings, attention, etc.).